In [ ]:
import torch
from torch import cuda
from torch.autograd import Variable
import torch.nn as nn
from transformers import BertModel, BertTokenizer, BertTokenizerFast
import datasets
from datasets import load_dataset
from torch.utils.data import DataLoader

# importing other required libraries
import argparse
import logging
import math
import dill
import os
import options
import random
import sys
import numpy as np
from collections import OrderedDict

import data
import utils
from meters import AverageMeter
from PGLoss import PGLoss
from tqdm import tqdm
from dictionary import Dictionary
import re
import subprocess
#Define the path of the cache directory
cache_dir = "~/.cache/huggingface/datasets"

#Define the name of the dataset
dataset_name = "wmt14"

# check if the dataset is already downloaded
if os.path.exists(os.path.join(cache_dir, dataset_name)):
    print("Dataset already downloaded") # if the dataset is already downloaded
else:
    print("Downloading the dataset")
    # Download the dataset
    dataset = load_dataset("wmt14", "fr-en")
    # dataset = datasets.load_dataset('wmt14', 'fr-en', cache_dir='path_to_cache_dir', download_mode='force_redownload') # if any error occurs, use this line of code
    # dataset = load_dataset("wmt14", "en-fr", split='train[:1%]', streaming=True) # Only loading 1% of train dataset for proto-typing purposes

# Load 50k rows of the train dataset
train_dataset = load_dataset("wmt14", "fr-en", split="train[:50000]")

# Keep the full valid and test datasets
valid_dataset = dataset["validation"]
test_dataset = dataset["test"]

# Loading Bert Model

bert_model = "bert-base-multilingual-cased"

In [36]:
def get_json_data(json_data):    
    en =list()
    fr = list()
    for row_idx in json_data:
        en.append(row_idx['translation']['en'])
        fr.append(row_idx['translation']['fr'])
    
    return en, fr

In [52]:
from transformers import BertTokenizer, AutoTokenizer
def preprocess(data):
    # Initialize the BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")
    # tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

    # print("data: ", data)
    en=list()
    fr=list()
    for element in data['translation']:
        # print("element: ", element)
        en.append(element['en'])
        fr.append(element['fr'] )
    # en, fr = get_json_data(data)
    
    # Tokenize the data
    inputs = tokenizer(
        en, truncation=True, padding="max_length", max_length=128
    )
    targets = tokenizer(
        fr, truncation=True, padding="max_length", max_length=128
    )

    # Convert tokens to their corresponding IDs
    input_ids = inputs.input_ids
    target_ids = targets.input_ids

    # Create attention masks
    input_attention_mask = inputs.attention_mask
    target_attention_mask = targets.attention_mask

    return {
        "input_ids": input_ids,
        "attention_mask": input_attention_mask,
        "target_ids": target_ids,
        "target_attention_mask": target_attention_mask,
    }

In [41]:
en, fr = get_json_data(train_dataset)

In [53]:
tokenized_train_datasets = train_dataset.map(
    preprocess, batched=True
)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map: 100%|██████████| 50000/50000 [02:40<00:00, 311.65 examples/s]


In [54]:
tokenized_train_datasets

Dataset({
    features: ['translation', 'input_ids', 'attention_mask', 'target_ids', 'target_attention_mask'],
    num_rows: 50000
})

In [55]:
from generator_tf_bert import TransformerModel_bert

In [56]:
generator2_train = TransformerModel_bert(args, use_cuda=use_cuda)

NameError: name 'args' is not defined